In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import*
import pandas as pd

In [2]:
from collections import Counter
#used to find frequency in the list

In [3]:
train = pd.read_csv('trainbb.csv')
test = pd.read_csv('testbb.csv')
print(test.isnull().sum())
train.isnull().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2             72344
Product_Category_3            162562
dtype: int64


User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            173638
Product_Category_3            383247
Purchase                           0
dtype: int64

In [ ]:
train = train.interpolate(method='barycentric')
test = test.interpolate(method='barycentric')

C:\Users\Vishnu\Anaconda3\envs\py35\lib\site-packages\scipy\interpolate\polyint.py:511: RuntimeWarning: overflow encountered in multiply
  self.wi[:j] *= (self.xi[j]-self.xi[:j])


# DATA PREPROCESSING

In [ ]:
train.isnull().sum()
test.Stay_In_Current_City_Years.replace('4+','10')

In [182]:
print(train.isnull().sum())
print(test.isnull().sum())
print(train.shape)
print(test.shape)

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64
Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64
(4650, 11)
(3099, 10)


In [183]:
tr_x = train.iloc[0:4650,1:10]
te_x = test.iloc[0:3099,1:10]
tr_y = train.iloc[0:4650,10]
print(tr_y.head())
Counter(tr_x.Outlet_Type)
print(tr_x.head())

0    3735.1380
1     443.4228
2    2097.2700
4     994.7052
5     556.6088
Name: Item_Outlet_Sales, dtype: float64
   Item_Weight  Item_Fat_Content  Item_Visibility     Item_Type  Item_MRP  \
0        9.300                 1         0.016047         Dairy  249.8092   
1        5.920                 0         0.019278   Soft Drinks   48.2692   
2       17.500                 1         0.016760          Meat  141.6180   
4        8.930                 1         0.000000     Household   53.8614   
5       10.395                 0         0.000000  Baking Goods   51.4008   

   Outlet_Establishment_Year  Outlet_Size  Outlet_Location_Type  \
0                       1999          2.0                     1   
1                       2009          2.0                     3   
2                       1999          2.0                     1   
4                       1987          3.0                     3   
5                       2009          2.0                     3   

         Outlet_Typ

In [184]:
#function to replace missing values
def transcore(x):
    if x == "Baking Goods":
        return 0
    if x == "Breads":
        return 1
    if x == "Break fast":
        return 2
    if x == "Canned":
        return 3
    if x == "Dairy":
        return 4
    if x == "Frozen Foods":
        return 5
    if x == "Fruits and Vegetables":
        return 6
    if x == "Hard Drinks":
        return 7
    if x == "Health and Hygiene":
        return 8
    if x == "Household":
        return 9
    if x == "Meat":
        return 10
    if x == "Others":
        return 11
    if x == "Seafood":
        return 12
    if x == "Snack Foods":
        return 13
    if x == "Soft Drinks":
        return 14
    if x == "Starchy Foods":
        return 15
    if x == " ":
        return 16
    

In [185]:
def groupscore(x):
    if x == "Grocery Store":
        return 0
    if x == "Supermarket Type1":
        return 1
    if x == "Supermarket Type2":
        return 2
    if x == "Supermarket Type3":
        return 3

In [186]:
tr_x.columns

Index(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Establishment_Year', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [187]:
tr_x['New_Type'] = tr_x['Item_Type'].apply(transcore)
#axis=1 indicates column while axis=0 indicates row
tr_x.drop('Item_Type', axis=1,inplace=True)
te_x['New_Type'] = te_x['Item_Type'].apply(transcore)
te_x.drop('Item_Type', axis=1,inplace=True)
#inplace=True defines that drop operation should take place
#inplace=True defines that drop operation should take place

In [188]:
#ensuring that all data are pre-processed for mathematical applications
print(tr_x.dtypes)
print(te_x.dtypes)

Item_Weight                  float64
Item_Fat_Content               int64
Item_Visibility              float64
Item_MRP                     float64
Outlet_Establishment_Year      int64
Outlet_Size                  float64
Outlet_Location_Type           int64
Outlet_Type                   object
New_Type                     float64
dtype: object
Item_Weight                  float64
Item_Fat_Content               int64
Item_Visibility              float64
Item_MRP                     float64
Outlet_Establishment_Year      int64
Outlet_Size                  float64
Outlet_Location_Type           int64
Outlet_Type                   object
New_Type                     float64
dtype: object


In [189]:
#checking missing values in a dataframe
print(te_x.isnull().sum())
print(tr_x.isnull().sum())

Item_Weight                   0
Item_Fat_Content              0
Item_Visibility               0
Item_MRP                      0
Outlet_Establishment_Year     0
Outlet_Size                   0
Outlet_Location_Type          0
Outlet_Type                   0
New_Type                     42
dtype: int64
Item_Weight                   0
Item_Fat_Content              0
Item_Visibility               0
Item_MRP                      0
Outlet_Establishment_Year     0
Outlet_Size                   0
Outlet_Location_Type          0
Outlet_Type                   0
New_Type                     58
dtype: int64


In [190]:
tr_x['Market_Type'] = tr_x['Outlet_Type'].apply(groupscore)
#axis=1 indicates column while axis=0 indicates row
tr_x.drop('Outlet_Type', axis=1,inplace=True)
te_x['Market_Type'] = te_x['Outlet_Type'].apply(groupscore)
te_x.drop('Outlet_Type', axis=1,inplace=True)

In [191]:
print(tr_x.isnull().sum())
print(te_x.isnull().sum())

Item_Weight                   0
Item_Fat_Content              0
Item_Visibility               0
Item_MRP                      0
Outlet_Establishment_Year     0
Outlet_Size                   0
Outlet_Location_Type          0
New_Type                     58
Market_Type                   0
dtype: int64
Item_Weight                   0
Item_Fat_Content              0
Item_Visibility               0
Item_MRP                      0
Outlet_Establishment_Year     0
Outlet_Size                   0
Outlet_Location_Type          0
New_Type                     42
Market_Type                   0
dtype: int64


In [192]:
tr_x.New_Type.fillna(1,inplace=True)
te_x.New_Type.fillna(1,inplace=True)
te_x.isnull().sum()

Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_MRP                     0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
New_Type                     0
Market_Type                  0
dtype: int64

In [193]:
tr_x.shape

(4650, 9)

In [194]:
tr_y.head()

0    3735.1380
1     443.4228
2    2097.2700
4     994.7052
5     556.6088
Name: Item_Outlet_Sales, dtype: float64

In [195]:
tr_y.shape

(4650,)

# Checking Dimensionality

In [196]:
tr_x.isnull().sum()

Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_MRP                     0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
New_Type                     0
Market_Type                  0
dtype: int64

In [197]:
te_x.isnull().sum()

Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_MRP                     0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
New_Type                     0
Market_Type                  0
dtype: int64

In [198]:
reg = lm.LinearRegression()

# Regression Starts

In [199]:
reg.fit(tr_x,tr_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [200]:
reg.coef_

array([-3.75417745e+00, -1.41138092e+01, -1.67012800e+02,  1.63117969e+01,
        1.62112042e+01,  6.68961762e+01,  3.01301036e+01,  5.61081178e-01,
       -5.97791846e+02])

In [201]:
pred = reg.predict(te_x)

In [202]:
Outlet_sales = pred

In [203]:
#Sales is predicted and given as output in Outlet_sales
Outlet_sales

array([1796.90003874, 1890.9687302 ,  478.87413984, ..., 1105.14373081,
       2294.86672831, 2463.09528674])

In [204]:
import random
Outlet_sales.shape

(3099,)

In [23]:
Id = np.array(range(0,5681))
Id.shape

(5681,)

In [24]:
Sales = pd.DataFrame({'Id':Id,'Outlet_sales':Outlet_sales})
Sales

,Id,Outlet_sales
0,0,1605.192734
1,1,1241.941807
2,2,3567.911310
3,3,2311.349049
4,4,3825.349672
5,5,1799.882467
6,6,919.396973
7,7,1810.779071
8,8,1044.132354
9,9,2870.932158


In [25]:
Sales.to_csv("Martsales.csv", encoding='utf-8')

# End of Program